Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import math 
import re
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

Reading the CSV file and check some information about it

In [ ]:
df = pd.read_csv('glassdoorJobs_Input.csv', keep_default_na=True )

In [ ]:
# Display the datatypes for each column
print(df.dtypes)

In [ ]:
# Display the DataFrame information
df.info()

In [ ]:
# Display the DataFrame statistics 
df.describe()

 Function to convert the indian currency to american currency

In [ ]:
def convert_to_american_format(indian_currency):

    # Check if the value is NaN to convert to 0
    if isinstance(indian_currency, float) and math.isnan(indian_currency):
        indian_currency = 0
    
    # Check if the value is numeric  (float, int, Decimal, etc.)
    if isinstance(indian_currency, (float, int)):
        numeric_value = int(float(indian_currency))  # Converte para float e depois para int
    else:
        # Convert to String to remove currency symbol and commas
        indian_currency = str(indian_currency).replace("₹", "").replace(",", "")
        numeric_value = int(float(indian_currency))  # Convert to float before int
    
    # Format the number to american style (mthousands separated by comma)
    american_format = f"{numeric_value}"
    
    return f"{american_format}"

Function to remove special characters from the pay period

In [ ]:
def convert_payperiod(payperiod):
    payperiod = str(payperiod)
    if payperiod == '/yr (est.)':
        # Replace '/yr (est.)' by 'yearly'
        payperiod = re.sub(r"/yr \(est.\)", "yearly", payperiod)
    elif payperiod == '/mo (est.)':
        # Replace '/mo (est.)' by 'monthly'
        payperiod = re.sub(r"/mo \(est.\)", "monthly", payperiod)
    elif payperiod == '/hr (est.)':
        # Replace '/hr (est.)' by 'hourly'
        payperiod = re.sub(r"/hr \(est.\)", "hourly", payperiod)
    else:
        payperiod = "unknown"
    return payperiod



 Function to convert the hourly and monthly salary to yearly

In [ ]:
def convert_salary_to_yearly(salary, frequency):
    salary = salary.replace("$","")
    salary = salary.replace(",","")
    salary = int(salary)
    if frequency == "hourly":
        # Suppose 40 hours per week, 52 weeks per year
        return salary * 40 * 52
    elif frequency == "monthly":
        # Multiplying by 12 to convert to Yearly
        return salary * 12
    elif frequency == "yearly":
        # If annual salary, return original value
        return salary
    else:
        pass

Starting data transformation to replace NaN and '--' to 'Unknown' for cleaness purpose

In [ ]:
# invoke the convert_to_american_format function and create a new column with the result
#df.converted_salary = df.salary_avg_estimate.apply(convert_to_american_format)
df.salary_avg_estimate = df.salary_avg_estimate.apply(convert_to_american_format)

In [ ]:
print(df.dtypes)

In [ ]:
# invoke the convert_payperiod function and create a new column with the result
df.converted_payperiod = df.salary_estimate_payperiod.apply(convert_payperiod)

In [ ]:
# invoke the converted_salary function and create a new column with the result
df.converted_annually_salary = df.apply(lambda row: convert_salary_to_yearly(row.converted_salary, row.converted_payperiod), axis=1)

In [ ]:
# Replace rows with string value from NaN' to 'Unknown' for the fields below
df.company = df.company.fillna('Unknown')
df.job_description = df.job_description.fillna('Unknown')
df.salary_avg_estimate = df.salary_avg_estimate.fillna('Unknown')
df.salary_estimate_payperiod = df.salary_estimate_payperiod.fillna('Unknown')
df.company_size = df.company_size.fillna('Unknown')
df.company_founded = df.company_founded.fillna('Unknown')
df.employment_type = df.employment_type.fillna('Unknown')
df.industry = df.industry.fillna('Unknown')
df.sector = df.sector.fillna('Unknown')
df.revenue = df.revenue.fillna('Unknown / Non-Applicable')

In [ ]:
# Replace rows with string value from '--' to 'Unknown' for the fields below
df.industry = df.industry.replace('--', 'Unknown')
df.sector = df.industry.replace('--', 'Unknown')
df.company_founded = df.company_founded.replace('--', 'Unknown')

In [ ]:
# Converting the salary_avg_estimate column to INT to calculate purposes
df.salary_avg_estimate = df.salary_avg_estimate.astype(int)

In [ ]:
# Converting the df.salary_avg_estimate to a NumPy array
avgSalary = df.salary_avg_estimate.to_numpy()

In [ ]:
df

In [ ]:
# display the dataframe information after data wrangling
df.info()

In [ ]:
# display the dataframe statistics after data wrangling
df.describe()

Evaluating the rating grouped by the categorical columns such as Company Size, Company Revenue, Employment Type, and Sector

In [ ]:
df.groupby(by='revenue').mean()[['company_rating','culture_and_values_rating','senior_management_rating','work_life_balance_rating','comp_and_benefits_rating','career_opportunities_rating']].round(2).reset_index().nlargest(10, 'company_rating')

In [ ]:
df.groupby(by='company_size').mean()[['company_rating','culture_and_values_rating','senior_management_rating','work_life_balance_rating','comp_and_benefits_rating','career_opportunities_rating']].round(2).reset_index().nlargest(10, 'company_rating')

In [ ]:
df.groupby(by='employment_type').mean()[['company_rating','culture_and_values_rating','senior_management_rating','work_life_balance_rating','comp_and_benefits_rating','career_opportunities_rating']].round(2).reset_index().nlargest(10, 'company_rating')

In [ ]:
df.groupby(by='sector').mean()[['company_rating','culture_and_values_rating','senior_management_rating','work_life_balance_rating','comp_and_benefits_rating','career_opportunities_rating']].round(2).reset_index().nlargest(10, 'company_rating')

In [ ]:
df.groupby(by='location').mean()[['company_rating','culture_and_values_rating','senior_management_rating','work_life_balance_rating','comp_and_benefits_rating','career_opportunities_rating']].round(2).reset_index().nlargest(10, 'company_rating')

In [ ]:
#df.company_size.value_counts(normalize=False)
# Group and sort by 'company_size'
df.groupby('company_size').size().reset_index(name='count').sort_values(by='count', ascending=False)

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Jobs by Company Size')
plt.barh(df.company_size.value_counts().index, df.company_size.value_counts())
plt.xticks( rotation=45, ha='right' )
plt.gca().invert_yaxis()
plt.show;

In [ ]:
#df.revenue.value_counts(normalize=False)
# Group and sort by 'revenue'
df.groupby('revenue').size().reset_index(name='count').sort_values(by='count', ascending=False)

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Jobs by Company Revenue')
plt.barh(df.revenue.value_counts().index, df.revenue.value_counts())
plt.xticks( rotation=45, ha='right' )
plt.gca().invert_yaxis()
plt.show;

In [ ]:
# Group and sort by 'employment_type'
empType=df.groupby('employment_type').size().reset_index(name='count').sort_values(by='count', ascending=False)
empType

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Jobs by Employment Type')
plt.barh(df.groupby('employment_type').reset_index(name='count').sort_values(by='count', ascending=False), df.groupby('employment_type').reset_index(name='count').sort_values(by='count', ascending=False))
plt.xticks( rotation=45, ha='right' )
plt.gca().invert_yaxis()
plt.show;

In [ ]:
fig = px.funnel(empType, x='count', y='employment type', title='Jobs by Employment Type')
fig.update_layout(title={'x': 0.5}) # centralize the title
fig.show();

In [ ]:
# Group and sort by sector
sectorCount = df.groupby('sector').size().reset_index(name='count')

# Filter and sort the sector with counter equal or bigger than 10
sectorCount_filtered = sectorCount[sectorCount['count'] >= 10].sort_values(by='count', ascending=False)
sectorCount_filtered

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Jobs by Sector')
plt.barh(sectorCount_filtered, sectorCount_filtered)
plt.xticks( rotation=45, ha='right' )
plt.gca().invert_yaxis()
plt.show;

In [ ]:
fig = px.funnel(sectorCount_filtered, x='count', y='sector', title='Jobs by Sector (top 10)')
fig.update_layout(title={'x': 0.5}) # centralize the title
fig.show()

In [ ]:
# Group and sort by sector
roleCount = df.groupby('job_title').size().reset_index(name='count')

# Filter and sort the sector with counting equal or bigger than 10
roleCount_filtered = roleCount[roleCount['count'] >= 10].sort_values(by='count', ascending=False)
roleCount_filtered

In [ ]:
plt.figure( figsize=(15,6) )
plt.title('Jobs by Role')
plt.pie(
    roleCount_filtered.head(10),
    labels = roleCount_filtered.index[0:10],
    shadow=True,
    startangle=90,
    autopct='%1.1f%%'
);

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Jobs by Role')
plt.barh(roleCount_filtered, roleCount_filtered)
plt.xticks( rotation=45, ha='right' )
plt.gca().invert_yaxis()
plt.show;

In [ ]:
# Group and sort by Location
localCount = df.groupby('location').size().reset_index(name='count')

# Filter and sort the sector with counter equal or bigger than 10
localCountFiltered = localCount[localCount['count'] >= 10].sort_values(by='count', ascending=False)
localCountFiltered

Evaluating the Company rating by Sector, Company Size, Company Revenue, Employment Type and Location

In [ ]:
# Group and sort by sector
sectorRattingAvg = df.groupby(by='sector').mean()['company_rating'].reset_index(name='mean').round(2)

# Filter and sort the sector with counter equal or bigger than 10
sectorAvgFiltered = sectorRattingAvg[sectorRattingAvg['mean'] >= 4].sort_values(by='mean', ascending=False)
sectorAvgFiltered

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Jobs by Role')
plt.barh(sectorAvgFiltered.index, sectorAvgFiltered)
plt.xticks( rotation=45, ha='right' )
plt.gca().invert_yaxis()
plt.show();

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Jobs by Company Revenue')
plt.barh(df.revenue.value_counts().index, df.revenue.value_counts())
plt.xticks( rotation=45, ha='right' )
plt.gca().invert_yaxis()
plt.show();

In [ ]:
df.groupby(by='company_size').mean()['company_rating'].reset_index(name='mean').sort_values(by='mean', ascending=False).round(2)

In [ ]:
df.groupby(by='revenue').mean()['company_rating'].reset_index(name='mean').sort_values(by='mean', ascending=False).round(2)

In [ ]:
df.groupby(by='employment_type').mean()['company_rating'].reset_index(name='mean').sort_values(by='mean', ascending=False).round(2)

Descritive Statistics: correlation, mean, median, mode, variance and standard deviation

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(), annot=True);

In [ ]:
# Converting the df.salary_avg_estimate to a NumPy array
avgSalary = df.salary_avg_estimate.to_numpy()

In [ ]:
# Mean
#print(f'Mean is {df.salary_avg_estimate.mean():.2f}')
print(f'Mean is {np.mean(avgSalary):.2f}')

In [ ]:
# Median
#print(f'Median is {df.salary_avg_estimate.median():.2f}')
print(f'Median is {np.median(avgSalary):.2f}')

In [ ]:
# Mode
print(f'Mode is {df.salary_avg_estimate.mode()}')

In [ ]:
# St Deviation
#print(f'Standard Deviation is {df.salary_avg_estimate.std():.2f}')
print(f'Standard Deviation is {np.std(avgSalary):.2f}')

In [ ]:
# Variance
#print(f'Variance is {df.salary_avg_estimate.var():.2f}')
print(f'Variance is {np.var(avgSalary):.2f}')

In [ ]:
sns.scatterplot(data=df, x='company_rating', y='career_opportunities_rating')
plt.title("Correlation between Company Rating and Career Opportunities Rating");

In [ ]:
sns.scatterplot(data=df, x='company_rating', y='culture_and_values_rating')
plt.title("Correlation between Company Rating and Culture and Values Rating");

In [ ]:
sns.scatterplot(data=df, x='company_rating', y='comp_and_benefits_rating')
plt.title("Correlation between Company Rating and Comp and Benefits Rating");

In [ ]:
sns.scatterplot(data=df, x='company_rating', y='senior_management_rating')
plt.title("Correlation between Company Rating and Senior Management Rating");

In [ ]:
sns.scatterplot(data=df, x='company_rating', y='work_life_balance_rating')
plt.title("Correlation between Company Rating and Work-Life Balance Rating");

In [ ]:
plt.figure(figsize=(8, 6))
sns.regplot(data=df, x='company_rating', y='work_life_balance_rating', scatter_kws={'s': 50})  # scatter_kws ajusta o tamanho dos pontos
plt.title("Correlation between Company Rating and Work-Life Balance Rating")
plt.show()

In [ ]:
# save the Dataframe content to a CSV file 
df.to_csv('glassdoorJobs_Output.csv')